In [652]:
import random
import sys
import time

In [653]:
def getNum(v) :
    n = len(v)
    index = random.randint(0, n - 1)
    num = v[index]
    v[index], v[n - 1] = v[n - 1], v[index]
    v.pop()
    return num

def GenerateRandomChromosome(NumberOfQueens):
    randomGenerator = [0] * NumberOfQueens
    
    chromosome = []
    for i in range(NumberOfQueens):
        randomGenerator[i] = i + 1

    while (len(randomGenerator)) :
        chromosome.append(getNum(randomGenerator)) 
        #getting the position of queens on board and storing it in chromosome

    #Example of generated output of this function [1,4,2,3,6,7,5,8] for 8 Queen
    return chromosome

In [654]:
def fitness(chromosome):
    #Function calculates the number of left and right side diagonal collisions.
    diagonalCollisons = 0 

    n = len(chromosome)
    leftDiagonal = [0] * 2*n
    rightDiagonal = [0] * 2*n
    for i in range(n):
        leftDiagonal[i + chromosome[i] - 1] += 1
        rightDiagonal[len(chromosome) - i + chromosome[i] - 2] += 1

    diagonalCollisons = 0
    for i in range(2*n-1):
        counter = 0
        if leftDiagonal[i] > 1:
            counter += leftDiagonal[i]-1
        if rightDiagonal[i] > 1:
            counter += rightDiagonal[i]-1
        diagonalCollisons += counter
    return int(maxFitness - diagonalCollisons) #28-(2+3)=23 
    #maxfitness is reduced by the number of collisions
    #therefore, the one with maximum colisions will have least fitness value

In [655]:
def parentSelection(chromosomeWithFitness):
    sortedPopulation = sorted(chromosomeWithFitness, key=lambda x: x['Fitness'], reverse=True)
    return [sortedPopulation[0], sortedPopulation[1]]

In [656]:
def crossover(parents):
    
    parent1 = parents[0]['Chromosome']
    parent2 = parents[1]['Chromosome']


    size = len(parent1)

    child1, child2 = [-1] * size, [-1] * size
    start, end = sorted([random.randrange(size) for _ in range(2)]) 
    #generating two random points 
    # and 
    # values between these two points will be copied directly to child

    child1Inherited = []
    child2Inherited = []
    for i in range(start, end + 1):
        child1[i] = parent1[i]
        child2[i] = parent2[i]
        child1Inherited.append(parent1[i])
        child2Inherited.append(parent2[i])

    #values are copied from the respective parents

    current_parent2_position, current_parent1_position = 0, 0

    fixed_pos = list(range(start, end + 1))       
    i = 0
    while i < size:
        if i in fixed_pos:
            i += 1
            continue

        testChild1 = child1[i]
        if testChild1==-1: #to be filled
            parent2Trait = parent2[current_parent2_position]
            while parent2Trait in child1Inherited:
                current_parent2_position += 1
                parent2Trait = parent2[current_parent2_position]
            child1[i] = parent2Trait
            child1Inherited.append(parent2Trait)

        i +=1

    #now remaining values are copied from the other parent 
    # example; 
    # we can see above that for child1 parent 2 values are copied 
        
    fixed_pos = list(range(start, end + 1))       
    i = 0
    while i < size:
        if i in fixed_pos:
            i += 1
            continue

        testChild2 = child2[i]
        if testChild2==-1:
            parent1Trait = parent1[current_parent1_position]
            while parent1Trait in child2Inherited:
                current_parent1_position += 1
                parent1Trait = parent1[current_parent1_position]
            child2[i] = parent1Trait
            child2Inherited.append(parent1Trait)

        i +=1
    #Similarly for child2 we copy from parent1
    return [child1, child2]
    

In [657]:
def mostCollision(chromosome):
    max = -sys.maxsize - 1
    n = len(chromosome)
    leftAttack = [0] * 2*n
    rightAttack = [0] * 2*n
    for i in range(n):
        value = 0
        leftAttack[i + chromosome[i] - 1] += 1
        rightAttack[len(chromosome) - i + chromosome[i] - 2] += 1
        value = leftAttack[i + chromosome[i] - 1]
        value += rightAttack[len(chromosome) - i + chromosome[i] - 2]
        if value > max:
            max = value
            maxIndex = i

    return maxIndex

In [658]:
def mutate(childs):
    start1 = mostCollision(childs[0])
    start2 = mostCollision(childs[1])
    end1, end2 = [random.randrange(len(childs[0])) for _ in range(2)]
    childs[0][start1], childs[0][end1] = childs[0][end1], childs[0][start1]
    childs[1][start2], childs[1][end2] = childs[1][end2], childs[1][start2]

    return childs

In [666]:
if __name__ == "__main__":
    start = time.time()
    numberOfQueens = 150 #Input: Number of queens
    maxFitness = (numberOfQueens*(numberOfQueens-1))/2 # (8*(8-1))/2 = 28 
    population = [GenerateRandomChromosome(numberOfQueens) for _ in range(1000)]
    generation = 1
    chromosomeWithFitness = []
    for n in population:
        tempDict = {'Chromosome': n}
        tempDict['Fitness'] = fitness(n)
        chromosomeWithFitness.append(tempDict)
    print("Number Queens: ", numberOfQueens)
    print(max([chrom['Fitness'] for chrom in chromosomeWithFitness]))
    
while maxFitness not in [chrom['Fitness'] for chrom in chromosomeWithFitness]:
    parents = parentSelection(chromosomeWithFitness)

    childs = crossover(parents)

    childs = mutate(childs)

    #replace the old population with new ones.
    chromosomeWithFitness = sorted(chromosomeWithFitness, key=lambda x: x['Fitness'])
    chromosomeWithFitness[0], chromosomeWithFitness[1] = {'Chromosome': childs[0], 'Fitness': fitness(childs[0])}, {'Chromosome': childs[1], 'Fitness': fitness(childs[1])}

    generation += 1

finalList = sorted(chromosomeWithFitness, key=lambda x: x['Fitness'], reverse=True)
print("Optimal Solution Chromosome: ", finalList[0]) #Output is number of generations and the N queen solution
print("Total Time to execute: ",time.time()-start)
print("Total Iterations/Generations: ",generation)

Number Queens:  150
11117
Optimal Solution Chromosome:  {'Chromosome': [32, 117, 40, 75, 112, 148, 66, 27, 43, 136, 91, 17, 88, 86, 67, 45, 47, 58, 138, 7, 57, 55, 70, 114, 13, 25, 83, 44, 121, 139, 5, 59, 133, 129, 2, 109, 60, 80, 94, 124, 132, 141, 125, 10, 96, 149, 111, 142, 90, 64, 35, 115, 103, 123, 134, 99, 63, 135, 36, 127, 1, 9, 12, 110, 20, 104, 42, 81, 14, 74, 120, 16, 19, 128, 46, 39, 31, 69, 101, 48, 18, 102, 22, 146, 21, 89, 56, 116, 147, 50, 33, 4, 105, 119, 76, 26, 38, 54, 82, 53, 140, 92, 73, 84, 87, 24, 118, 29, 52, 145, 49, 144, 107, 65, 11, 131, 93, 68, 23, 137, 79, 3, 130, 113, 98, 6, 106, 34, 30, 122, 51, 150, 95, 143, 37, 15, 85, 97, 100, 8, 126, 61, 108, 41, 72, 78, 62, 71, 77, 28], 'Fitness': 11175}
Total Time to execute:  1.782418966293335
Total Iterations/Generations:  1167
